# Main Problem

In [58]:
import json

In [59]:
def get_transport_matrix():
    BASE_PATH = "../data/routeData/"
    daytimes = [("morning", 8), ("midday", 12), ("evening", 16), ("night", 20)]
    transport_matrix = {}

    for time_in_day, hour in daytimes:
        filepath = BASE_PATH + f"route_matrix_{time_in_day}.json"

        with open(filepath, 'r') as f:
            route_matrix = json.load(f)

        for route in route_matrix["routes"]:
            this_route = route_matrix["routes"][route]
            origin = this_route["origin_name"]
            destination = this_route["destination_name"]
            transport_matrix[(origin, destination, hour)] = {
                "transit": {
                    "duration": this_route["transit"]["duration_minutes"],
                    "price": this_route["transit"]["fare_sgd"],
                },
                "drive": {
                    "duration": this_route["drive"]["duration_minutes"],
                    "price": this_route["drive"]["fare_sgd"],
                }
            }
    
    return transport_matrix


def get_all_locations():
    BASE_PATH = "../data/routeData/"

    with open(BASE_PATH + "route_matrix_morning.json", 'r') as f:
        route_matrix = json.load(f)
    
    locations = [route_matrix["locations"][location_id] for location_id in route_matrix["locations"]]
    return locations


def get_transport_hour(transport_time):
    # because the transport_matrix is only bracketed to 4 groups, we find the earliest it happens
    brackets = [8, 12, 16, 20]
    transport_hour = transport_time // 60

    for bracket in reversed(brackets):
        if transport_hour >= bracket:
            return bracket

    return brackets[-1] # from 8 PM to 8 AM next day


In [60]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

In [61]:
np.random.seed(42)

In [243]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import os
import logging

# from utils.transport_utility import get_transport_hour

# Set up logging
os.makedirs("log", exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("log/generated_problem.log", mode='a'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("generated_problem")

class TravelItineraryProblem(ElementwiseProblem):

    def validate_inputs(self, budget, locations, transport_matrix, num_days):
        """Validate input data and print warnings/errors"""
        logging.info("Validating optimization inputs...")
        
        # Check if we have at least one hotel
        hotels = [loc for loc in locations if loc["type"] == "hotel"]
        if not hotels:
            logging.error("No hotels found in locations data!")
        else:
            logging.info(f"Found {len(hotels)} hotels in the data")
        
        # Check if we have hawkers for meals
        hawkers = [loc for loc in locations if loc["type"] == "hawker"]
        if not hawkers:
            logging.error("No hawker centers found - meal constraints cannot be satisfied!")
        else:
            logging.info(f"Found {len(hawkers)} hawker centers in the data")
        
        # Check for attractions
        attractions = [loc for loc in locations if loc["type"] == "attraction"]
        logging.info(f"Found {len(attractions)} attractions in the data")
        
        # Validate location data completeness
        for i, loc in enumerate(locations):
            missing = []
            if loc["type"] == "attraction":
                if "entrance_fee" not in loc or loc["entrance_fee"] is None:
                    missing.append("entrance_fee")
                if "satisfaction" not in loc or loc["satisfaction"] is None:
                    missing.append("satisfaction")
                if "duration" not in loc or loc["duration"] is None:
                    missing.append("duration")
            elif loc["type"] == "hawker":
                if "rating" not in loc or loc["rating"] is None:
                    missing.append("rating")
                if "duration" not in loc or loc["duration"] is None:
                    missing.append("duration")
            
            if missing:
                logging.warning(f"Location '{loc['name']}' is missing required fields: {', '.join(missing)}")
        
        # Check transport matrix completeness
        sample_routes = 0
        missing_routes = 0
        for i, src in enumerate(locations):
            for j, dest in enumerate(locations):
                if i != j:  # Skip self-routes
                    for hour in [8, 12, 16, 20]:  # Time brackets
                        key = (src["name"], dest["name"], hour)
                        if key not in transport_matrix:
                            missing_routes += 1
                            if missing_routes <= 5:  # Only log the first few missing routes
                                logging.error(f"Missing transport data: {key}")
                        else:
                            sample_routes += 1
        
        if missing_routes > 0:
            logging.error(f"Missing {missing_routes} routes in transport matrix!")
        else:
            logging.info(f"Transport matrix contains all required routes ({sample_routes} total)")
        
        # Check budget feasibility
        hotel_cost = num_days * 50  # Using your fixed HOTEL_COST of 50
        min_food_cost = num_days * 2 * 10  # Minimum 2 meals per day at 10 each
        
        min_cost = hotel_cost + min_food_cost
        if budget < min_cost:
            logging.error(f"Budget ({budget}) is too low! Minimum needed is {min_cost} for hotel and food alone")
        else:
            logging.info(f"Budget check passed: {budget} >= minimum {min_cost} for hotel and food")
    
    def __init__(self, budget, locations, transport_matrix, num_days=3):
        
        # Add validation checks before setup
        self.validate_inputs(budget, locations, transport_matrix, num_days)
        
        # constants
        self.NUM_DAYS = num_days
        self.HOTEL_COST = 50
        self.START_TIME = 9 * 60 # everyday starts from 9 AM
        self.HARD_LIMIT_END_TIME = 22 * 60 # everyday MUST return back to hotel by 10 PM

        # Define lunch and dinner time windows (in minutes since start of day at 9 AM)
        self.LUNCH_START = 11 * 60  # 11 AM (9 AM + 2 hours)
        self.LUNCH_END = 15 * 60    # 3 PM (9 AM + 6 hours)
        self.DINNER_START = 17 * 60  # 5 PM (9 AM + 8 hours)
        self.DINNER_END = 21 * 60   # 9 PM (9 AM + 12 hours)
        
        # hard limit money spent
        self.budget = budget
        # list of destinations
        self.locations = locations
        self.num_locations = len(locations)
        self.num_attractions = len([loc for loc in self.locations if loc["type"] == "attraction"])
        self.num_hawkers = len([loc for loc in self.locations if loc["type"] == "hawker"])
        self.num_hotels = 1 # always 1

        # transportation option prices and durations. The size MUST BE num_dest * num_dest, for 24 hours
        self.transport_types = ["transit", "drive"]
        self.num_transport_types = len(self.transport_types)
        self.transport_matrix = transport_matrix

        # x_ijkl = BINARY VAR, if the route goes in day i, use transport type j, from location k, to location l
        self.x_shape = self.NUM_DAYS * self.num_transport_types * self.num_locations * self.num_locations
        # u_ik  = CONTINUOUS VAR, tracking FINISH time of the person in day i, location k
        self.u_shape = self.NUM_DAYS * self.num_locations

        num_vars = self.x_shape + self.u_shape

        lower_bound = np.concatenate([np.zeros(self.x_shape), np.full(self.u_shape, 0)])
        upper_bound = np.concatenate([np.ones(self.x_shape), np.full(self.u_shape, self.HARD_LIMIT_END_TIME)])
        
        # Count constraints
        def calculate_constraints():
            ### For counting actual inequality constraints
            g_count = 0
            
            # For each attraction, must be visited at most once as source and at most once as destination
            g_count += 2 * self.num_attractions

            # For each hawker everyday, must be visited at most once as source and at most once as destination
            g_count += 2 * self.NUM_DAYS * self.num_hawkers

            # Every day, go from hotel at least START_TIME
            g_count += self.NUM_DAYS
            
            # For time constraints when a route is chosen
            g_count += self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1)
            g_count += self.NUM_DAYS * self.num_transport_types * (self.num_locations - 1) * (self.num_locations - 2)
            
            # For hawker visits (at least twice per day)
            g_count += self.NUM_DAYS
            
            # For lunch and dinner time constraints
            g_count += self.NUM_DAYS * 2
            
            # For transport type constraints (can't use both transit and drive for the same route)
            g_count += self.NUM_DAYS * self.num_locations * self.num_locations
            
            # Budget constraint
            g_count += 1
            
            # Min/max total visits constraints
            g_count += 2
            
            ### For equality constraints
            h_count = 0
            
            # Flow conservation (in = out)
            h_count += self.NUM_DAYS * self.num_locations
            
            # Hotel must be starting point each day
            h_count += self.NUM_DAYS
            
            # Return to hotel constraint
            h_count += self.NUM_DAYS
            
            # If attraction is visited as source, it must be visited as destination
            h_count += self.NUM_DAYS * self.num_locations
            
            return g_count, h_count

        # Calculate actual constraints
        num_inequality_constraints, num_equality_constraints = calculate_constraints()
        
        super().__init__(
            n_var=num_vars,
            n_obj=3, # INEQUALITY_CONSTRAINT_LINE
            n_ieq_constr=num_inequality_constraints,
            n_eq_constr=num_equality_constraints,
            xl=lower_bound,
            xu=upper_bound,
        )
        
    def test_feasibility(self):
        """Test if the problem has any feasible solutions"""
        logging.info("Testing problem feasibility...")
        
        # Check if we have enough hawkers for lunch and dinner every day
        if self.num_hawkers == 0:
            logging.error("Infeasible: No hawker centers available for meals")
            return False
        
        # Check if we can meet the time constraints
        # This is a simplified check - minimum time would be:
        # - Start at hotel
        # - Travel to lunch hawker
        # - Eat lunch (60 min)
        # - Travel to attraction 
        # - Visit attraction
        # - Travel to dinner hawker
        # - Eat dinner (60 min)
        # - Travel back to hotel
        
        # Check if there's enough time in the day for this minimum itinerary
        available_time = self.HARD_LIMIT_END_TIME - self.START_TIME  # Minutes available
        logging.info(f"Available time per day: {available_time} minutes")
        
        # Simple feasibility test on time windows 
        lunch_window = self.LUNCH_END - self.LUNCH_START
        dinner_window = self.DINNER_END - self.DINNER_START
        logging.info(f"Lunch window: {lunch_window} minutes, Dinner window: {dinner_window} minutes")
        
        # Check if we can satisfy hawker constraints 
        if lunch_window < 60:
            logging.error(f"Infeasible: Lunch window ({lunch_window} min) too short for a 60 min meal")
            return False
        
        if dinner_window < 60:
            logging.error(f"Infeasible: Dinner window ({dinner_window} min) too short for a 60 min meal")
            return False
        
        return True

    def get_transport_hour(self, transport_time):
        # because the transport_matrix is only bracketed to 4 groups, we find the earliest it happens
        brackets = [8, 12, 16, 20]
        transport_hour = transport_time // 60

        for bracket in reversed(brackets):
            if transport_hour >= bracket:
                return bracket

        return brackets[-1] # from 8 PM to 8 AM next day
    
    def _evaluate(self, x, out, *args, **kwargs):
        # they're ensured to be integers
        x_var = x[:self.x_shape].reshape(self.NUM_DAYS, self.num_transport_types, self.num_locations, self.num_locations)
        u_var = x[self.x_shape:].reshape(self.NUM_DAYS, self.num_locations)
        # x_var = np.array([[[
        #         [ 0,  1,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  1,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  1],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  1,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0]],

        #         [[ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 1,  0,  0,  0,  0,  0,  0,  0]]],

        #         [[[ 0,  0,  0,  0,  0,  0,  0,  1],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  1,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 1,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  1,  0,  0,  0,  0]],

        #         [[ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0]]],

        #         [[[ 0,  0,  0,  0,  0,  0,  1,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  1],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  1,  0,  0,  0],
        #         [ 1,  0,  0,  0,  0,  0,  0,  0]],

        #         [[ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0],
        #         [ 0,  0,  0,  0,  0,  0,  0,  0]]]])

        # u_var = np.array([[ 540,  642,  830,    0,    0,    0,  726,  1080],
        #                   [ 540,    0,    0,  825,    0,    0,  1080,  720],
        #                   [ 540,    0,    0,    0,  850,    0,  720,  1080]])

        # initialize constraints
        # equality constraints
        out["H"] = []
        # inequality constraints
        out["G"] = []

        total_cost = self.NUM_DAYS * self.HOTEL_COST  # Cost starts with hotel cost for number of nights
        total_travel_time = 0
        total_satisfaction = 0

        # for every attraction, must be a source & destination at most once
        # NOTE that this doesn't apply to hawkers
        for k in range(self.num_locations):
            if self.locations[k]["type"] == "attraction":
                out["G"].append(np.sum(x_var[:, :, k, :]) - 1)
                out["G"].append(np.sum(x_var[:, :, :, k]) - 1)
        
        # for hawkers, every day must be a source & destination at most once
        for i in range(self.NUM_DAYS):
            for k in range(self.num_locations):
                if self.locations[k]["type"] == "hawker":
                    out["G"].append(np.sum(x_var[i, :, k, :]) - 1)
                    out["G"].append(np.sum(x_var[i, :, :, k]) - 1)
                
                # If attraction is a source, it must also be a destination
                out["H"].append(np.sum(x_var[:, :, k, :]) - np.sum(x_var[:, :, :, k]))
                # if len(out["H"]) in [26, 28, 30]:
                #     print(f"{len(out['H'])}: NUM OF CHOSEN TO BE SOURCE == DEST {np.sum(x_var[:, :, k, :])} {np.sum(x_var[:, :, :, k])}")

        for i in range(self.NUM_DAYS):
            # u_var[i, 0] must be the smallest of u_var[i]
            non_zero_elements = u_var[i, u_var[i, :] > 0]
            if len(non_zero_elements) > 0:
                out["H"].append(np.min(non_zero_elements) - u_var[i, 0])
            else: # if everything is zero, this is violated already
                out["H"].append(-1)
            # if len(out["H"]) in [26, 28, 30]:
            #     print(f"{len(out['H'])}: HOTEL MUST BE STARTING")

            # day start must be 8 AM or more
            out["G"].append((self.START_TIME - u_var[i, 0]))

            for j in range(self.num_transport_types):
                for k in range(self.num_locations):

                    for l in range(1, self.num_locations): # NOTE here that hotel (index 0) isn't included in destination. It will be computed later
                        if k == l: continue
                        # every day, if the route is chosen, the time of finishing in this place is this

                        # if chosen, then time_should_finish_l must be time_finish_l, otherwise, don't matter (just put 0)
                        # you should finish attraction l at:
                        #   - time to finish k + 
                        #   - time to transport from k to l, using transport method j +
                        #   - time to play at l
                        time_finish_l = u_var[i, l]
                        transport_hour = self.get_transport_hour(u_var[i, k])
                        transport_value = self.transport_matrix[(self.locations[k]["name"], self.locations[l]["name"], transport_hour)][self.transport_types[j]]
                        time_should_finish_l = u_var[i, k] + transport_value["duration"] + self.locations[l]["duration"]
                        # if x_var[i, j, k, l] is not chosen, then this constraint don't matter
                        out["G"].append(x_var[i, j, k, l] * (time_should_finish_l - time_finish_l))
                        # if len(out["G"]) in [62]:
                        #     print(f"{len(out['G'])}: XIJ CONSTRAINT, {i} {j} {k} {l}, {x_var[i, j, k, l]} {time_should_finish_l} {time_finish_l}")
                        
                        # append to total travel time spent
                        if x_var[i, j, k, l] == 1:
                            # calculate the travel
                            total_travel_time += transport_value["duration"]
                            total_cost += transport_value["price"]

                            # calculate cost and satisfaction, based on what they come to
                            # Note: not counting anything for hotel.
                            if self.locations[l]["type"] == "attraction":
                                total_cost += self.locations[l]["entrance_fee"]
                                total_satisfaction += self.locations[l]["satisfaction"]
                            elif self.locations[l]["type"] == "hawker":
                                total_cost += 10 # ASSUME eating in a hawker is ALWAYS $10
                                total_satisfaction += self.locations[l]["rating"]

            # from last place, return to hotel.
            last_place = np.argmax(u_var[i, :])
            # THIS SHOULD NEVER HAPPEN: if last_place is already the hotel, then get the second last
            if last_place == 0: # doesn't make sense, because the first constraint is u[i, 0] must be the smallest (it's starting point)
                last_place = np.argsort(u_var[i, :])[-2]
                latest_hour = self.get_transport_hour(u_var[i, last_place])
                out["H"].append(np.sum(x_var[i, :, last_place, 0]) - 1)
            else:
                latest_hour = self.get_transport_hour(u_var[i, last_place])
                # MUST go back to hotel at the end of the day
                out["H"].append(np.sum(x_var[i, :, last_place, 0]) - 1)
                # if len(out["H"]) in [26, 28, 30]:
                #     print(f"{len(out['H'])}: day {i}, last place {last_place} must be going to hotel last {x_var[i, :, last_place, 0]}")
                # go back to hotel
                # choose whether to use which transport type
                for j in range(self.num_transport_types):
                    if x_var[i, j, last_place, 0]:
                        # pull from google maps to get the transport details if the decision is here.
                        gohome_value = self.transport_matrix[(self.locations[last_place]["name"], self.locations[0]["name"], latest_hour)][self.transport_types[j]]
                        total_travel_time += gohome_value["duration"]
                        total_cost += gohome_value["price"]
                        break

        for i in range(self.NUM_DAYS):
            
            lunch_hawker_visit = 0
            dinner_hawker_visit = 0
            
            hawker_sum = 0
            for k in range(self.num_locations):
                # every day, for every location, if it's selected as source, it must be selected as destination
                out["H"].append(np.sum(x_var[i, :, :, k]) - np.sum(x_var[i, :, k, :]))
                # if len(out["H"]) in [26, 28, 30]:
                #     print(f"{len(out['H'])}: LOCATION OF SOURCE & DESTINATION {i} {k}, {np.sum(x_var[i, :, :, k])} - {np.sum(x_var[i, :, k, :])}")

                if self.locations[k]["type"] == "hawker":
                    hawker_sum += np.sum(x_var[i, :, k, :])
                    
                    # For each route ending at this hawker, check if it's during lunch time or dinner time
                    for src in range(self.num_locations):
                        if src == k:
                            continue
                        for j_transport in range(self.num_transport_types):
                            if u_var[i, k] >= self.LUNCH_START and u_var[i, k] <= self.LUNCH_END:
                                lunch_hawker_visit += x_var[i, j_transport, src, k]
                            
                            if u_var[i, k] >= self.DINNER_START and u_var[i, k] <= self.DINNER_END:
                                dinner_hawker_visit += x_var[i, j_transport, src, k]

            # every day, must go to hawkers at least twice (lunch & dinner. Can go more times if they want to)
            out["G"].append(2 - hawker_sum)
            
            # every day, must visit a hawker during lunch time (at least one hawker visit with arrival/stay during lunch hours)
            if self.num_hawkers > 0:  # Only add constraint if there are hawkers available
                out["G"].append(1 - lunch_hawker_visit)
                # if len(out["G"]) in [62]:
                #     print(f"{len(out['G'])}: LUNCH HAWKER VISIT, {min_visits} {np.sum(x_var)}")
                
                # every day, must visit a hawker during dinner time (at least one hawker visit with arrival/stay during dinner hours)
                out["G"].append(1 - dinner_hawker_visit)
                # if len(out["G"]) in [62]:
                #     print(f"{len(out['G'])}: DINNER HAWKER VISIT")

        for i in range(self.NUM_DAYS):
            for k in range(self.num_locations):
                for l in range(self.num_locations):
                    # for every day, if public transportation is chosen, taxi can't be chosen, and vice versa
                    # (sum j in transport_types x_ijkl <= 1) for all days, for all sources, for all destinations
                    out["G"].append(np.sum(x_var[i, :, k, l]) - 1)
                    # if len(out["G"]) in [62]:
                    #     print(f"{len(out['G'])}: PUBLIC TRANSPORT & TAXI MAX 1 day {i}, from {k}, to {l}")

        # finally, make sure everything is within budget
        out["G"].append(total_cost - self.budget)
        # if len(out["G"]) in [166]:
        #     print(f"{len(out['G'])}: OVERBUDGET, budget {self.budget} total cost{total_cost}")
        
        # Calculate reasonable minimum and maximum visits
        # Minimum: At least 2 hawker visits per day (lunch & dinner)
        min_visits = self.NUM_DAYS * 2
        # Maximum: Reasonable upper bound for visits per day (e.g., start at hotel + 2 meals + 2-3 attractions)
        max_visits_per_day = 6
        max_visits = self.NUM_DAYS * max_visits_per_day

        # Constraint: Total visits should be at least the minimum
        out["G"].append(min_visits - np.sum(x_var))
        # if len(out["G"]) in [166]:
        #     print(f"{len(out['G'])}: MINIMUM VISIT CONSTRAINT, {min_visits} {np.sum(x_var)}")
        # Constraint: Total visits should be at most the maximum
        out["G"].append(np.sum(x_var) - max_visits)
        # if len(out["G"]) in [166]:
        #     print(f"{len(out['G'])}: MAXIMUM VISIT CONSTRAINT, {max_visits} {np.sum(x_var)}")
        # INDENTATION_COUNT_LINE
        # <ADD ADDITIONAL CONSTRAINTS HERE>

        # objectives
        out["F"] = [total_cost, total_travel_time, -total_satisfaction]

        # print("OUT G", out["G"])
        # print("G violated constraints:", [(idx, g) for idx, g in enumerate(out["G"]) if g > 0])
        # print("OUT H", out["H"])
        # print("H violated constraints:", [(idx, h) for idx, h in enumerate(out["H"]) if h != 0])


In [244]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.optimize import minimize
from pymoo.termination import get_termination

In [245]:
all_locations = get_all_locations()

# for all locations, get satisfaction and rating
for loc in all_locations:
    if loc["type"] == "hawker":
        loc["rating"] = np.random.randint(0, 5)
        loc["avg_food_price"] = np.random.randint(5, 15)
        loc["duration"] = 60 # just standardize 60 mins
    elif loc["type"] == "attraction":
        loc["satisfaction"] = np.random.randint(0, 10)
        loc["entrance_fee"] = np.random.randint(5, 100)
        loc["duration"] = np.random.randint(30, 90)

dummy_hotel = {
    "type": "hotel",
    "name": "DUMMY HOTEL",
    "lat": 1.2852044,
    "lng": 103.8610313,
}

In [246]:
transport_matrix = get_transport_matrix()
# add dummy hotel to transport_matrix
for loc in all_locations:
    time_brackets = [8, 12, 16, 20]
    for time_ in time_brackets:
        transport_matrix[(dummy_hotel["name"], loc["name"], time_)] = {
            "transit": {
                "duration": 50,
                "price": 1.93,
            },
            "drive": {
                "duration": 20,
                "price": 10.1,
            }
        }
        transport_matrix[(loc["name"], dummy_hotel["name"], time_)] = {
            "transit": {
                "duration": 50,
                "price": 1.93,
            },
            "drive": {
                "duration": 20,
                "price": 10.1,
            }
        }

locations = [dummy_hotel] + all_locations

In [247]:
import random
random.seed(42)

hotels = [loc for loc in locations if loc["type"] == "hotel"]
attractions = [loc for loc in locations if loc["type"] == "attraction"]
hawkers = [loc for loc in locations if loc["type"] == "hawker"]

selected_hotel = random.sample(hotels, 1)
selected_attractions = random.sample(attractions, 5)
selected_hawkers = random.sample(hawkers, 2)

selected_locations = selected_hotel + selected_attractions + selected_hawkers

In [248]:
selected_locations

[{'type': 'hotel',
  'name': 'DUMMY HOTEL',
  'lat': 1.2852044,
  'lng': 103.8610313},
 {'id': 'A4',
  'name': 'ArtScience Museum',
  'type': 'attraction',
  'lat': 1.2862738,
  'lng': 103.8592663,
  'satisfaction': 2,
  'entrance_fee': 29,
  'duration': 42},
 {'id': 'A7',
  'name': 'National Museum of Singapore',
  'type': 'attraction',
  'lat': 1.296613,
  'lng': 103.8485091,
  'satisfaction': 9,
  'entrance_fee': 87,
  'duration': 81},
 {'id': 'A13',
  'name': 'Esplanade',
  'type': 'attraction',
  'lat': 1.2897934,
  'lng': 103.8558166,
  'satisfaction': 9,
  'entrance_fee': 48,
  'duration': 58},
 {'id': 'A11',
  'name': 'Clarke Quay',
  'type': 'attraction',
  'lat': 1.2906024,
  'lng': 103.8464742,
  'satisfaction': 2,
  'entrance_fee': 22,
  'duration': 33},
 {'id': 'A12',
  'name': 'Orchard Road',
  'type': 'attraction',
  'lat': 1.3048205,
  'lng': 103.8321984,
  'satisfaction': 7,
  'entrance_fee': 48,
  'duration': 84},
 {'id': 'H3',
  'name': 'Hill Street Tai Hwa Pork Nood

In [249]:
for loc1 in selected_locations:
    for loc2 in selected_locations:
        if loc1 == loc2:
            continue
        for time_ in [8, 12, 16, 20]:
            print(f"Transport FROM {loc1['name']} TO {loc2['name']} at {time_} =", transport_matrix[(loc1["name"], loc2["name"], time_)])
        print()

Transport FROM DUMMY HOTEL TO ArtScience Museum at 8 = {'transit': {'duration': 50, 'price': 1.93}, 'drive': {'duration': 20, 'price': 10.1}}
Transport FROM DUMMY HOTEL TO ArtScience Museum at 12 = {'transit': {'duration': 50, 'price': 1.93}, 'drive': {'duration': 20, 'price': 10.1}}
Transport FROM DUMMY HOTEL TO ArtScience Museum at 16 = {'transit': {'duration': 50, 'price': 1.93}, 'drive': {'duration': 20, 'price': 10.1}}
Transport FROM DUMMY HOTEL TO ArtScience Museum at 20 = {'transit': {'duration': 50, 'price': 1.93}, 'drive': {'duration': 20, 'price': 10.1}}

Transport FROM DUMMY HOTEL TO National Museum of Singapore at 8 = {'transit': {'duration': 50, 'price': 1.93}, 'drive': {'duration': 20, 'price': 10.1}}
Transport FROM DUMMY HOTEL TO National Museum of Singapore at 12 = {'transit': {'duration': 50, 'price': 1.93}, 'drive': {'duration': 20, 'price': 10.1}}
Transport FROM DUMMY HOTEL TO National Museum of Singapore at 16 = {'transit': {'duration': 50, 'price': 1.93}, 'drive': 

In [252]:
problem = TravelItineraryProblem(
    num_days=3,
    budget=1000,
    locations=selected_locations,
    transport_matrix=transport_matrix,
)

algorithm = NSGA2(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=TwoPointCrossover(),
    mutation=BitflipMutation(),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 500)

# Test if the problem has all required attributes
required_attrs = ['n_var', 'n_obj', 'n_ieq_constr', 'n_eq_constr', '_evaluate']
missing_attrs = [attr for attr in required_attrs if not hasattr(problem, attr)]
if missing_attrs:
    logger.error(f"Problem is missing required attributes: {missing_attrs}")
    
# After creating the problem but before running optimization
problem.test_feasibility()

# You might want to add an option to exit after debugging
debug_only = False  # Set to True to exit after debugging
if debug_only:
    logger.info("Debug mode only, exiting before optimization")

res = minimize(
    problem,
    algorithm,
    termination,
    seed=1,
    save_history=True,
    verbose=True
)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

# Save the best solution to a file
# np.save("results/best_solution.npy", res.X)
# np.save("results/best_objectives.npy", res.F)
logger.info("Best solution and objectives saved to results directory")

2025-03-21 12:23:20,072 - INFO - Validating optimization inputs...
2025-03-21 12:23:20,072 - INFO - Found 1 hotels in the data
2025-03-21 12:23:20,073 - INFO - Found 2 hawker centers in the data
2025-03-21 12:23:20,073 - INFO - Found 5 attractions in the data
2025-03-21 12:23:20,074 - INFO - Transport matrix contains all required routes (224 total)
2025-03-21 12:23:20,074 - INFO - Budget check passed: 1000 >= minimum 210 for hotel and food
2025-03-21 12:23:20,075 - INFO - Testing problem feasibility...
2025-03-21 12:23:20,075 - INFO - Available time per day: 780 minutes
2025-03-21 12:23:20,076 - INFO - Lunch window: 240 minutes, Dinner window: 240 minutes


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  3.084280E+04 |  4.583912E+04 |             - |             -
     2 |      200 |      1 |  2.736142E+04 |  3.976946E+04 |             - |             -
     3 |      300 |      1 |  2.736142E+04 |  3.622624E+04 |             - |             -
     4 |      400 |      1 |  2.736142E+04 |  3.342357E+04 |             - |             -
     5 |      500 |      1 |  2.673161E+04 |  3.158813E+04 |             - |             -
     6 |      600 |      1 |  2.673161E+04 |  2.961652E+04 |             - |             -
     7 |      700 |      1 |  2.148049E+04 |  2.811603E+04 |             - |             -
     8 |      800 |      1 |  2.148049E+04 |  2.687079E+04 |             - |             -
     9 |      900 |      1 |  2.148049E+04 |  2.552066E+04 |             - |             -
    10 |     1000 |      1 |  1.982460E+04 |  2.408679E+04 |             - |             -

2025-03-21 12:24:29,330 - INFO - Best solution and objectives saved to results directory


   500 |    50000 |      1 |  1.782217E+03 |  1.783976E+03 |             - |             -
Best solution found: 
X = None
F = None


In [251]:
res.X